In [ ]:
import os
import sys
import time
import glob
import re
import json

import pandas as pd
import numpy as np

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

from torchvision import models, transforms

from accelerate import Accelerator

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import accuracy_score, f1_score

import seaborn as sns
from matplotlib import pyplot as plt

from tqdm import tqdm

import cv2
from PIL import Image, ImageChops

import zipfile

# Dataset

In [ ]:
class ImageBinaryDataset(Dataset):
    def __init__(self, image_dir = 'TriKotaiNarkota/TriKotaiNarkota', transform=None):
        
        self.image_dir = image_dir
        if transform is None:
            self.transform = transforms.Compose([
                transforms.Resize((256, 256)),  # Изменяем размер до 256x256
                # transforms.Grayscale(num_output_channels=1),
                transforms.ToTensor()           # Преобразуем в тензор с диапазоном [0, 1]
            ])
        else:
            self.transform = transform

        # Columns for binary labels
        self.label_columns = [
            "Некачественное ГДИС", "Влияние ствола скважины", "Радиальный режим",
            "Линейный режим", "Билинейный режим", "Сферический режим",
            "Граница постоянного давления", "Граница непроницаемый разлом"
        ]

        self.image_folders = glob.glob(os.path.join(self.image_dir, "sample*"))
        self.pairs = []
        for img_folder in tqdm(self.image_folders):
            labels = np.load(os.path.join(img_folder, 'labels.npy'))
            labels = labels # Оставляем только бинарные признаки
            self.pairs.append((img_folder, labels))

        # Compute class weights to handle imbalance
        self.class_weights = self.compute_class_weights()

    def compute_class_weights(self):
        """ Computes inverse class frequencies to use in BCEWithLogitsLoss """
        labels = np.array([pair[1][:8] for pair in self.pairs])  # Extract labels
        pos_counts = np.sum(labels, axis=0)  # Count occurrences of each class
        neg_counts = labels.shape[0] - pos_counts  # Count negatives per class

        pos_weight = neg_counts / (pos_counts + 1e-6)  # Avoid division by zero

        return torch.tensor(pos_weight, dtype=torch.float32)
    
    def __len__(self):
        return len(self.image_folders)

    def __getitem__(self, idx):
        img_folder_path, labels = self.pairs[idx]

        image_plot = Image.open(os.path.join(img_folder_path, 'plot.png')).convert("RGB")
        
        image_scatter = Image.open(os.path.join(img_folder_path, 'scatter.png')).convert("RGB")

        torch_plot = self.transform(image_plot)
        torch_scatter = self.transform(image_scatter)
    
        res_img = torch.cat([torch_plot, torch_scatter], dim = 0)

        if len(labels) > 15:
            labels, starts_values = labels[:15], labels[15:]
            return res_img, torch.tensor(labels, dtype=torch.float32), torch.tensor(starts_values, dtype=torch.float32)
            
        return res_img, torch.tensor(labels, dtype=torch.float32)

## Create dataset and dataloader

In [ ]:
transformer = transforms.Compose([
    transforms.Resize((256, 256)),  # Изменяем размер до 256x256
    # transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor()           # Преобразуем в тензор с диапазоном [0, 1]
])

In [ ]:
dataset = ImageBinaryDataset('TriKotaiNarkota/TriKotaiNarkota')

len(dataset)

100%|██████████| 257144/257144 [00:52<00:00, 4891.37it/s]


257144

In [ ]:
dataset[0][0].size(), dataset[0][1].size(), dataset[0][2].size()

(torch.Size([6, 256, 256]), torch.Size([15]), torch.Size([24]))

In [ ]:
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=150, shuffle=True, num_workers=16, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=150, shuffle=False, num_workers=16, pin_memory=True)

# Model

In [ ]:
markup_train = pd.read_csv('markup_train.csv')
hq_markup_train = pd.read_csv('hq_markup_train.csv')

len(markup_train), len(hq_markup_train)

(45141, 500)

In [ ]:
markup_train.describe()

,Некачественное ГДИС,Влияние ствола скважины,Радиальный режим,Линейный режим,Билинейный режим,Сферический режим,Граница постоянного давления,Граница непроницаемый разлом,Влияние ствола скважины_details,Радиальный режим_details,Линейный режим_details,Билинейный режим_details,Сферический режим_details,Граница постоянного давления_details,Граница непроницаемый разлом_details
count,45141.000000,45141.000000,45141.000000,45141.000000,45141.000000,45141.000000,45141.000000,45141.000000,35558.000000,25855.000000,12211.000000,12086.000000,5764.000000,4874.000000,4265.000000
mean,0.160984,0.787710,0.572761,0.270508,0.267739,0.127689,0.107973,0.094482,1.719234,1.140486,0.605509,0.934330,1.292040,144.019966,160.928583
std,0.367521,0.408934,0.494683,0.444227,0.442786,0.333747,0.310349,0.292501,1.537100,1.155191,1.012616,1.166539,0.982978,782.593841,339.370082
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-3.843521,-3.707246,-4.109357,-3.332898,-5.164659,0.006653,0.010753
25%,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.914535,0.504870,0.028459,0.327374,0.743594,18.335495,29.301152
50%,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.589366,0.959171,0.516372,0.720093,1.296814,55.182021,75.822331
75%,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,2.256645,1.422883,1.024682,1.130444,1.808614,134.488126,182.395620
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,31.777765,6.408605,29.027806,29.129381,6.631359,51035.492870,11595.502940


In [ ]:
markup_train.iloc[:, -7:].quantile([0.05, 0.95])

,Влияние ствола скважины_details,Радиальный режим_details,Линейный режим_details,Билинейный режим_details,Сферический режим_details,Граница постоянного давления_details,Граница непроницаемый разлом_details
0.05,-0.456332,-0.289947,-0.696055,-0.344091,-0.212926,2.517347,4.377187
0.95,5.495232,4.063072,2.154472,3.984768,2.709264,494.381095,554.024368


## Normalize

In [ ]:
RANGES = [
    (-0.5, 5.5),   # Влияние ствола скважины_details
    (-0.3, 4.1),   # Радиальный режим_details
    (-0.7, 2.2),   # Линейный режим_details
    (-0.4, 4.0),   # Билинейный режим_details
    (-0.3, 3.0),   # Сферический режим_details
    (2.4,  495.0),  # Граница постоянного давления_details
    (4.0,  555.0)   # Граница непроницаемый разлом_details
]

# Функция нормализации
def normalize_labels(num_labels):
    num_labels_scaled = num_labels.clone()  # Создаём копию
    for i in range(7):
        min_val, max_val = RANGES[i]
        mask = num_labels[:, i] != -1000  # Только для валидных значений
        num_labels_scaled[mask, i] = (num_labels[mask, i] - min_val) / (max_val - min_val)
    return num_labels_scaled.clip(0, 1)  # Обрезаем значения до [0,1]

# Функция обратного преобразования
def denormalize_labels(pred_labels):
    pred_labels_orig = pred_labels.clone()
    for i in range(7):
        min_val, max_val = RANGES[i]
        pred_labels_orig[:, i] = pred_labels[:, i] * (max_val - min_val) + min_val
    return pred_labels_orig

## Create Model

In [ ]:
# Определение модели
class MultiTaskResNet(nn.Module):
    def __init__(self, pretrained=False):
        super().__init__()
        self.base = models.resnet101(pretrained=pretrained)

        # Преобразуем первый слой под 2 канала
        orig_conv = self.base.conv1
        new_conv = nn.Conv2d(
            in_channels=6,
            out_channels=orig_conv.out_channels,
            kernel_size=orig_conv.kernel_size,
            stride=orig_conv.stride,
            padding=orig_conv.padding,
            bias=orig_conv.bias
        )
        with torch.no_grad():
            new_conv.weight[:, :3, :, :] = orig_conv.weight[:, :3, :, :]
            new_conv.weight[:, 3:, :, :] = orig_conv.weight[:, :3, :, :]
        self.base.conv1 = new_conv

        # Финальный FC: 15 выходов (8 – бинарные, 7 – регрессия)
        num_feats = self.base.fc.in_features
        self.base.fc = nn.Identity()
        
        self.binary = nn.Sequential(
            nn.Linear(num_feats, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.1),
            
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(0.1),

            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(0.1),

            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(0.1),

            nn.Linear(64, 8),
        )

        self.resgresion = nn.Sequential(
            nn.Linear(num_feats + 24, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.1),

            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(0.1),

            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(0.1),

            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(0.1),

            nn.Linear(64, 7),
        )

    def forward(self, x, start_values  = None):
        out = self.base(x)
        
        logits_8 = self.binary(out)
        if start_values is None:
            regr_7   = torch.sigmoid(self.resgresion(torch.cat((out, torch.zeros(x.size()[0], 24)), axis = 1)))
        else:
            regr_7   =  torch.sigmoid(self.resgresion(torch.cat((out, start_values), axis = 1)))
        return logits_8, regr_7

In [ ]:
model = MultiTaskResNet()

/usr/lib/python3/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/lib/python3/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
model.load_state_dict(
    torch.load(
        'models/resnet18_300v5_0.9478026046710444_0.9995309066868887.pth',
        weights_only = True,
        map_location = 'cpu',
    )
)

<All keys matched successfully>

In [ ]:
# from accelerate import Accelerator
# accelerator = Accelerator(mixed_precision="fp16")

device = 'cuda'
model = model.to(device)

pos_weight = dataset.compute_class_weights().to(device)

criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = optim.Adam(model.parameters(), lr=0.001)

scheduler = StepLR(optimizer, step_size=8, gamma=0.4)

## multitask_loss_function

In [ ]:
pos_weight = dataset.compute_class_weights().to(device)

def multitask_loss_function(logits_8, regr_7, bin_labels_8, num_labels_7, alpha_mse=1.0, alpha_mae = 0.5):
    bce = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    
    loss_class = bce(logits_8, bin_labels_8)

    valid_mask = (num_labels_7 != -1000)

    phenomenon_present_mask = torch.zeros_like(valid_mask)
    for i in range(7):
        phenomenon_present_mask[:, i] = (bin_labels_8[:, i+1] > 0.5)

    final_mask = valid_mask & phenomenon_present_mask

    regr_active = regr_7[final_mask]
    gt_active   = num_labels_7[final_mask]

    if regr_active.numel() > 0:
        mse = nn.MSELoss()(regr_active, gt_active)
        mae = nn.L1Loss()(regr_active, gt_active)
    else:
        mse = torch.tensor(0.0, device=logits_8.device)
        mae = torch.tensor(0.0, device=logits_8.device)

    total_loss = loss_class + alpha_mse * mse + alpha_mae * mae
    return total_loss, loss_class.item(), mse.item(), mae.item()

## custom_binary_accuracy


In [ ]:
def custom_binary_accuracy(y_true, y_pred):
    
    correct_presence = (y_true == 1) & (y_pred == 1)  
    correct_absence = (y_true == 0) & (y_pred == 0) 
    
    correct_predictions = np.sum(correct_presence) + np.sum(correct_absence)
    total_samples = y_true.size  

    return correct_predictions / total_samples

## custom_binary_accuracy_xlop

In [ ]:
def case_score(bin_true, bin_pred, num_true, num_pred):
    y_tp_0 = np.expand_dims((bin_pred[:, 0] == 1) & (bin_true[:, 0] == 1), axis = 1)
    y_tp_1_7 = (bin_pred[:, 1:] == 1) & (bin_true[:, 1:] == 1) & (abs(num_pred-num_true) <= 0.15)
    
    y_tp = np.concatenate((y_tp_0, y_tp_1_7), axis = 1)
    y_tp = np.sum(y_tp)
    
    y_fp_0 = np.expand_dims((bin_pred[:, 0] == 1) & (bin_true[:, 0] == 0), axis = 1)
    y_fp_1_7 = (bin_pred[:, 1:] == 1) & ((bin_true[:, 1:] == 0) | (abs(num_pred-num_true) > 0.15))
    
    y_fp = np.concatenate((y_fp_0, y_fp_1_7), axis = 1)
    y_fp = np.sum(y_fp)
    
    y_fn_0 = np.expand_dims((bin_pred[:, 0] == 0) & (bin_true[:, 0] == 1), axis = 1)
    y_fn_1_7 = ((bin_pred[:, 1:] == 0) | (abs(num_pred-num_true) > 0.15)) & (bin_true[:, 1:] == 1)

    y_fn = np.concatenate((y_fn_0, y_fn_1_7), axis = 1)
    y_fn = np.sum(y_fn)
    
    press = y_tp / (y_tp + y_fp)
    
    recall = y_tp / (y_tp + y_fn)
    
    f1 = 2 * press * recall / (press + recall)
    return f1

# Train Loop

In [ ]:
name='resnet18_300v5'
num_epochs=25
alpha_mse= 1
alpha_mae = 1

Изменения:
- Добавил две головы в модель
- Для расчета регрессионных фичей используются начальные данные, они добавленны в датасет, и подаеются в модель
- Данные не нормируются, так как становится не понятно реальное mse, mae


In [ ]:
torch.cuda.empty_cache()

In [ ]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    mse_epoch_loss = 0.0
    mae_epoch_loss = 0.0
    bce_epoch_loss = 0.0
    for images, labels, start_values in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Training]"):
        model.to(device)
        images       = images.to(device)  # shape [B, 2, H, W]
        start_values = start_values.to(device)
        bin_labels   = labels[:, 0:8].to(device)  # Binary classification labels
        num_labels   = labels[:, 8:15].to(device) # shape [B, 7]

        # **Нормализация числовых меток**
        num_labels = normalize_labels(num_labels) 

        optimizer.zero_grad()

        # Forward pass
        logits_8, regr_7 = model(images, start_values)
 
        total_loss, bce_val, mse_val, mae_val = multitask_loss_function(
            logits_8, regr_7, bin_labels, num_labels, alpha_mse=alpha_mse, alpha_mae = alpha_mae
        )

        # Backprop
        total_loss.backward()
        optimizer.step()

        running_loss   += total_loss.item() * images.size(0)
        mse_epoch_loss += mse_val * images.size(0)
        mae_epoch_loss += mae_val * images.size(0)
        bce_epoch_loss += bce_val * images.size(0)
        torch.cuda.empty_cache()

    scheduler.step()
    running_loss   /= len(train_loader.dataset)
    mse_epoch_loss /= len(train_loader.dataset)
    mae_epoch_loss /= len(train_loader.dataset)
    bce_epoch_loss /= len(train_loader.dataset)

    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {running_loss:.6f} (BCE={bce_epoch_loss:.4f}, MSE={mse_epoch_loss:.4f}, MAE={mae_epoch_loss:.4f})")

    # **Валидация**
    model.eval()
    with torch.no_grad():
        all_preds_bin = []
        all_true_bin  = []
        all_preds_num = []
        all_true_num  = []
        all_presence_mask = []

        for images, labels, start_values in tqdm(test_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Validation]"):
            images       = images.to(device)
            start_values = start_values.to(device)
            bin_labels   = labels[:, 0:8].to(device)  # Binary classification labels
            num_labels   = labels[:, 8:15].to(device)

            # **Нормализация**
            # num_labels = normalize_labels(num_labels)

            logits_8, regr_7 = model(images, start_values)

            # Классификация
            pred_bin = torch.sigmoid(logits_8)
            pred_bin = (pred_bin > 0.5).float()

            all_preds_bin.append(pred_bin.cpu())
            all_true_bin.append(bin_labels.cpu())

            # Регрессия: **обратное масштабирование**
            regr_7_orig = denormalize_labels(regr_7)

            all_preds_num.append(regr_7_orig.cpu())
            all_true_num.append(num_labels.cpu())  # Не забудь тоже вернуть в оригинал при анализе!
            presence_mask = ((bin_labels[:, 1:] > 0.5) & (num_labels != -1000)).cpu()
            all_presence_mask.append(presence_mask)

        # **Анализ предсказаний**
        all_preds_bin = torch.cat(all_preds_bin, dim=0).numpy()
        all_true_bin  = torch.cat(all_true_bin, dim=0).numpy()

        val_acc = accuracy_score(all_true_bin, all_preds_bin)
        f1_macro = f1_score(all_true_bin, all_preds_bin, average="macro")

        # **Обратная денормализация на финальном этапе**
        all_preds_num = torch.cat(all_preds_num, dim=0).numpy()
        all_true_num  = torch.cat(all_true_num, dim=0).numpy()
        all_presence_mask = torch.cat(all_presence_mask, dim=0).numpy()

        within_10pct_count = 0
        total_present = 0
        for i in range(all_true_num.shape[0]):
            for j in range(all_true_num.shape[1]):
                if all_presence_mask[i, j]:
                    gt = all_true_num[i, j]
                    pr = all_preds_num[i, j]
                    rel_err = abs(pr - gt) / (abs(gt) + 1e-9)
                    if rel_err <= 0.13:
                        within_10pct_count += 1
                    total_present += 1
        numeric_within_10pct = within_10pct_count / total_present if total_present > 0 else 0.0

        print(f"Val Accuracy: {val_acc:.4f}, Val Macro-F1: {f1_macro:.4f}")
        print(f"Val Numeric Within ±10%: {numeric_within_10pct:.4f}")

        binary_score      = custom_binary_accuracy(all_true_bin, all_preds_bin)
        binary_score_xlop = custom_binary_accuracy_xlop(all_true_bin, all_preds_bin, all_true_num, all_preds_num)
        numeric_score     = numeric_within_10pct  
        final_score       = 0.7 * binary_score + 0.3 * numeric_score
        print(f"Binary score: {binary_score:.4f}")
        print(f'Case score: {binary_score_xlop:.4f}')
        print(f"Final score: {final_score:.4f}")

    torch.save(model.state_dict(), f"models/{name}_{final_score}_{binary_score}.pth")

Epoch 1/25 [Training]: 100%|██████████| 1372/1372 [27:35<00:00,  1.21s/it]


Epoch 1/25 - Loss: 0.063018 (BCE=0.0418, MSE=0.0009, MAE=0.0203)


Epoch 1/25 [Validation]: 100%|██████████| 343/343 [01:45<00:00,  3.26it/s]


Val Accuracy: 0.8147, Val Macro-F1: 0.9579
Val Numeric Within ±10%: 0.5567
Binary score: 0.9742
Case score: 0.5806
Final score: 0.8490


Epoch 2/25 [Training]: 100%|██████████| 1372/1372 [27:38<00:00,  1.21s/it]


Epoch 2/25 - Loss: 0.061182 (BCE=0.0403, MSE=0.0009, MAE=0.0200)


Epoch 2/25 [Validation]: 100%|██████████| 343/343 [01:44<00:00,  3.28it/s]


Val Accuracy: 0.8896, Val Macro-F1: 0.9804
Val Numeric Within ±10%: 0.6262
Binary score: 0.9853
Case score: 0.6571
Final score: 0.8775


Epoch 3/25 [Training]:  81%|████████  | 1105/1372 [22:15<05:20,  1.20s/it]